In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 0.Before Start

### 0.1 Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 0.2 Loading Data

In [ ]:
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cats=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

ID-테스트 세트 내에서 (Shop, Item) 튜플을 나타내는 Id\
shop_id-상점의 고유 식별자\
item_id-상품의 고유 식별자\
item_category_id-항목 카테고리의 고유 식별자\
item_cnt_day-판매 된 제품 수입니다. 이 측정 값의 월별 금액을 예측하고 있습니다.\
item_price-상품의 현재 가격\
date-dd / mm / yyyy 형식의 날짜\
date_block_num-편의를 위해 사용되는 연속 월 번호입니다. 2013 년 1 월은 0, 2013 년 2 월은 1, ..., 2015 년 10 월은 33입니다.\
item_name-항목 이름\
shop_name-상점 이름\
item_category_name-항목 카테고리 이름

In [ ]:
test.head()

In [ ]:
items.head()

In [ ]:
item_cats.head()

In [ ]:
shops.head()

In [ ]:
submission.head()

In [ ]:
train.head()

In [ ]:
print("shape of train:",train.shape)
print("shape of test:",test.shape)
print("shape of submission:",submission.shape)
print("shape of items:",items.shape)
print("shape of item_cats:",item_cats.shape)
print("shape of shops:",shops.shape)

## 1. Features engineering

### 1.1 drop duplicates, remove train which is not in test

In [ ]:
# drop duplicates 
subset = ['date','date_block_num','shop_id','item_id','item_cnt_day'] 
print(train.duplicated(subset=subset).value_counts()) 
train.drop_duplicates(subset=subset, inplace=True)
print("shape of train:",train.shape)

In [ ]:
# drop shops&items not in test data 
test_shops = test.shop_id.unique() 
test_items = test.item_id.unique() 
train = train[train.shop_id.isin(test_shops)] 
train = train[train.item_id.isin(test_items)] 
print("shape of train:",train.shape)

### 1.2 Checking outlier

In [ ]:
fig = plt.figure(figsize=(18,9))
plt.subplots_adjust(hspace=.5)

plt.subplot2grid((3,3), (0,0), colspan = 3)
train['shop_id'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Shop ID Values in the Training Set (Normalized)')

plt.subplot2grid((3,3), (1,0))
train['item_id'].plot(kind='hist', alpha=0.7)
plt.title('Item ID Histogram')

plt.subplot2grid((3,3), (1,1))
train['item_price'].plot(kind='hist', alpha=0.7, color='orange')
plt.title('Item Price Histogram')

plt.subplot2grid((3,3), (1,2))
train['item_cnt_day'].plot(kind='hist', alpha=0.7, color='green')
plt.title('Item Count Day Histogram')

plt.subplot2grid((3,3), (2,0), colspan = 3)
train['date_block_num'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Month (date_block_num) Values in the Training Set (Normalized)')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))

plt.subplot2grid((1,2), (0,0))
train['item_price'].plot(kind='box')
plt.title('Item price')

plt.subplot2grid((1,2), (0,1))
train['item_cnt_day'].plot(kind='box')
plt.title('item_cnt_day')

plt.show()

> ### 1.3 Feature Creation

month

In [ ]:
train['month']=train['date_block_num']

In [ ]:
train['month']+=1
train['month']%=12
print('shape of train',train.shape)

In [ ]:
train['month']

season

In [ ]:
#season
train['season']=train['month']

In [ ]:
train['season']//=3

In [ ]:
train['season']=train['season'].replace(0,4)

area

In [ ]:
#area
train=pd.merge(train,shops,how='left',on='shop_id')

In [ ]:
area = train['shop_name'].apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0]) 
train['area'] = pd.Categorical(area).codes 
train.head()

In [ ]:
train.shape

discount rate

In [ ]:
from itertools import product

block_shop_combi = pd.DataFrame(list(product(np.arange(34), test_shops)), columns=['date_block_num','shop_id']) 
shop_item_combi = pd.DataFrame(list(product(test_shops, test_items)), columns=['shop_id','item_id']) 
all_combi = pd.merge(block_shop_combi, shop_item_combi, on=['shop_id'], how='inner')

In [ ]:
train_base = pd.merge(all_combi, train, on=['date_block_num','shop_id','item_id'], how='left') 
train_base['item_cnt_day'].fillna(0, inplace=True) 
train_grp = train_base.groupby(['date_block_num','shop_id','item_id'])

In [ ]:
train_month = pd.DataFrame(train_grp.agg({'item_cnt_day':['sum','count']})).reset_index() 
train_month.columns = ['date_block_num','shop_id','item_id','item_cnt','item_order']
train_month.head()

In [ ]:
grp = train_month.groupby(['shop_id', 'item_id']) 
train_shop = grp.agg({'item_cnt':['mean','median','std'],'item_order':'mean'}).reset_index() 
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_med_shop','cnt_std_shop','order_mean_shop'] 
print(train_shop[['cnt_mean_shop','cnt_med_shop','cnt_std_shop']].describe())
train_shop.head()

In [ ]:
price_max = train.groupby(['item_id']).max()['item_price'].reset_index()
price_max.rename(columns={'item_price':'item_max_price'}, inplace=True)
price_max.head()

In [ ]:
price = train.groupby(['item_id']).mean()['item_price'].reset_index()
price.rename(columns={'item_price':'item_mean_price'}, inplace=True)
price.head()

In [ ]:
train_price_dc = pd.merge(price, price_max, on=['item_id'], how='left') 
train_price_dc['discount'] = 1 - (train_price_dc['item_mean_price'] / train_price_dc['item_max_price']) 
train_price_dc.drop('item_max_price', axis=1, inplace=True) 
train_price_dc.head()

In [ ]:
train=pd.merge(train,train_price_dc,how='left',on='item_id')
train.head()

shop_item_id

In [ ]:
train['shop_id']=train['shop_id'].astype(str)
train['item_id']=train['item_id'].astype(str)
train['shop_item']=train['shop_id']+"-"+train['item_id']
train

month

In [ ]:
train_month['shop_id']=train_month['shop_id'].astype(str)
train_month['item_id']=train_month['item_id'].astype(str)
train_month['shop_item']=train_month['shop_id']+"-"+train_month['item_id']

In [ ]:
del train_month['date_block_num']
del train_month['shop_id']
del train_month['item_id']
del train_month['item_order']

In [ ]:
train=pd.merge(train,train_month,on='shop_item')

In [ ]:
train

## 2. Modeling

### 2.1 SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
X = train.iloc[:, 0:12]
Y = train.iloc[:, 12]
x_train, x_test, y_train, y_test = train_test_split(X, Y, stratify = Y, random_state = 0)

In [ ]:
svc = SVC(C = 1000, gamma = 0.00001)
svc.fit(x_train, y_train)

In [ ]:
print("prediction :", svc.predict(x_test))
print("train accuracy :", svc.score(x_train, y_train))
print("test accuracy :", svc.score(x_test, y_test))

### 2.2 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=10, random_state=0)
forest.fit(X_train, y_train)

print("prediction :", forest.predict(X_test))
print("train accuracy :", forest.score(X_train, y_train))
print("test accuracy :", forest.score(X_test, y_test))

### 2.3 Gradient Boosting Regression Tree

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=0,max_depth=1)

gbrt.fit(X_train,y_train)


print("train accuracy :".format(gbrt.score(X_train,y_train)))
print("test accuracy :".format(gbrt.score(X_test,y_test)))